In [ ]:
import pandas as pd
import numpy as np
import math
from goodreads import client, shelf
import oauth2 as oauth
import urllib
import collections

## Get authentication oauth2

In [ ]:
url = 'http://www.goodreads.com'
request_token_url = '%s/oauth/request_token/' % url
authorize_url = '%s/oauth/authorize/' % url
access_token_url = '%s/oauth/access_token/' % url

In [ ]:
gc = client.GoodreadsClient('63pG3gIBwuO4vq83YxHpw', 'p81lgA89eoowbbkVtsh7hdoB2fdW5ELeXijb2Kk')

In [ ]:
consumer = oauth.Consumer(key='63pG3gIBwuO4vq83YxHpw',
                          secret='P4t2Nvwesy7h4DG9hTKdBbdQHlo76Uz7Nxd6iFJZU')


oclient = oauth.Client(consumer)

response, content = oclient.request(request_token_url, 'GET')
if response['status'] != '200':
    raise Exception('Invalid response: %s ' % response['status'])

request_token = dict(urllib.parse.parse_qsl(content))

authorize_link = '%s?oauth_token=%s' % (authorize_url,
                                        request_token['oauth_token'])

authorize_link

## Get Titles from selected authors

In [ ]:
df_bx_book = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Books.csv', sep=';', encoding='iso-8859-15', error_bad_lines=False)
df_bx_book.head()

In [ ]:
df_node_authors = pd.read_csv('/home/ignacio/Datasets/Graph analysis/authors_nodes.csv')
df_node_authors.head()

In [ ]:
df_bx_book = df_bx_book[ df_bx_book['Book-Author'].isin( df_node_authors['Authors'] ) ]
df_bx_book.head()

In [ ]:
len(df_bx_book)

In [ ]:
df_bx_book[df_bx_book['ISBN']=='1566191432']

In [ ]:
df_bx_book.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Books-Subset.csv', index=False)

## Get shelves from GoodReads

In [ ]:
df_bx_book = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Books-Subset.csv')
#df_bx_book = df_bx_book[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
df_bx_book.head()

### Modify information based on GoodReads

In [ ]:
df_bx_book['ISBN'] = df_bx_book['ISBN'].str.replace('0515134503','9780515134506')
df_bx_book.head()

In [ ]:
df_bx_book = df_bx_book.drop([1123])

### Function to find shelves

In [ ]:
gc = client.GoodreadsClient('63pG3gIBwuO4vq83YxHpw', 'P4t2Nvwesy7h4DG9hTKdBbdQHlo76Uz7Nxd6iFJZU')

#### get popupar shelves

In [ ]:
def get_popular_shelves(book):
    if type(book._book_dict['popular_shelves']['shelf']) is collections.OrderedDict :
        return [shelf.GoodreadsShelf(book._book_dict['popular_shelves']['shelf'])]
    else:
        return [shelf.GoodreadsShelf(pop_shel) for pop_shel in book._book_dict['popular_shelves']['shelf']]
    

#### find shelves

In [ ]:
def find_shelves(row, isbn_list):
#    print(row['ISBN'])
    try:
        book = gc.book(isbn=row['ISBN'])
        result = get_popular_shelves(book)
    except Exception:
        print('ISBN '+ row['ISBN'] +' INVALID')
        isbn_list.append( row['ISBN'] )
        result = []
    return str(result)

In [ ]:
frame = 100
max_complete_frames = math.floor(len(df_bx_book)/frame)
max_complete_frames

In [ ]:
isbn_invalid = []

In [ ]:
for i in np.arange(125, max_complete_frames):
    df_test = df_bx_book.loc[i*frame : (i+1)*frame]
    df_test['shelves'] = df_test.apply(find_shelves, axis=1, isbn_list=isbn_invalid)
    df_test.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-book-shelves_'+ str(i) +'.csv', index=False)
    print("Iteration " + str(i) )

In [ ]:
if len(df_bx_book) % frame != 0:
    df_test = df_bx_book.loc[max_complete_frames*frame : len(df_bx_book) ]
    df_test['shelves'] = df_test.apply(find_shelves, axis=1, isbn_list=isbn_invalid)
    df_test.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-book-shelves_'+ str(max_complete_frames) +'.csv', index=False)

In [ ]:
df_book_shelves = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-book-shelves_0.csv')
for i in np.arange(1,max_complete_frames+1):
    df = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-book-shelves_'+str(i)+'.csv')
    df_book_shelves = pd.concat([df_book_shelves, df])
df_book_shelves = df_book_shelves.reset_index()
df_book_shelves.head()

In [ ]:
df_book_shelves = df_book_shelves.drop(axis=1, labels=['index'])
df_book_shelves.head()

In [ ]:
df_book_shelves.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-book-shelves.csv', index=False)

In [ ]:
pop_shelve = book._book_dict['popular_shelves']

In [ ]:
type(pop_shelve['shelf']) is collections.OrderedDict

In [ ]:
get_popular_shelves(book)

In [ ]:
for k, v in pop_shelve['shelf'].items():
    print(v)

In [ ]:
book2 = gc.book(isbn='2070360024')

In [ ]:
get_popular_shelves(book2)

In [ ]:
book2.average_rating

In [ ]:
pop_shelve2 = book2._book_dict['popular_shelves']
pop_shelve2['shelf']

In [ ]:
print(shelves_list)